<a href="https://colab.research.google.com/github/atuleshchand/PythonDataScience/blob/main/FruitsClassificaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
import pathlib
from zipfile import ZipFile
import PIL

import tensorflow as tf

import shutil


In [ ]:
zf = ZipFile("/content/drive/MyDrive/archive.zip")
zf.extractall("/content")

shutil.rmtree("/content/fruits-360-original-size/")

In [ ]:
train_dir = pathlib.Path('/content/fruits-360_dataset/fruits-360/Training')
test_dir = pathlib.Path('/content/fruits-360_dataset/fruits-360/Test')

In [ ]:
# Total images we have in dir
image_count = len(list(train_dir.glob("*/*.jpg")))
image_count

# Visualization

In [ ]:
fruits = list(train_dir.glob("Apple Red Delicious/*.jpg"))
plt.figure(figsize=(10,10))

for i in range(9):
  plt.subplot(3, 3, i + 1)
  img = PIL.Image.open(str(fruits[i]))
  plt.imshow(img)
  plt.axis('off')

plt.show()

In [ ]:
batch_size = 32
img_height = 100
img_width = 100

In [ ]:
# Get images and put them into tensorflow dataset

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
# Get images and put them into tensorflow dataset

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)

In [ ]:
for images, labels in train_ds.take(1):
  plt.figure(figsize=(10,10))
  for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(class_names[labels[i]])
    plt.axis('off')

  plt.show()

# Preprosessing/Setting Up Base Model

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
preprocess_input = tf.keras.applications.resnet.preprocess_input

In [ ]:
base_model = tf.keras.applications.resnet.ResNet50(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

In [ ]:
base_model.trainable = False

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(num_classes)

# Building the Model

In [ ]:
inputs = tf.keras.Input(shape=(100, 100, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
model.summary()

# Training the model

In [ ]:
model.evaluate(val_ds)

In [ ]:
epochs = 10

history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(12, 10))
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper left')
plt.title('Training and Validation Loss')

plt.show()

In [ ]:
np.argmin(val_loss)